<a href="https://colab.research.google.com/github/ballmdr/Machine-Learning/blob/master/Forex_Prediction_Deep_Learning_Pure_Numpy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

from datetime import datetime

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
filename = '/content/drive/My Drive/export/M30_all_EURUSD.csv'
df = pd.read_csv(filename, names=['Date', 'Time', 'Open', 'High', 'Low', 'Close', 'Volume'])
df['Datetime'] = pd.to_datetime(df.Date + ' ' + df.Time)
df.set_index('Datetime', inplace=True)

# Features for Momentum Strategy
df['Returns'] = - df.Close.pct_change(-1)

df['MA35'] = df.Close.rolling(35).mean()
df['MA70'] = df.Close.rolling(70).mean()
df['MA200'] = df.Close.rolling(200).mean()

# ma35 > ma70 = bullish
df['35_70'] = np.where(df.MA35 >= df.MA70, 1, 0)

# price > ma200 = bullish
df['bias_trend'] = np.where(df.Close.values >= df.MA200, 1, 0) 

df['body_candles'] = df.Open - df.Close
df['high_low'] = df.High - df.Low

df.dropna(inplace=True)

def norm_features(x):
  return (x - x.mean()) / (x.max() - x.min())

# normalize
df['Returns'] = norm_features(df.Returns)
df['Volume'] = norm_features(df.Volume)
df['body_candles'] = norm_features(df.body_candles)
df['high_low'] = norm_features(df.high_low)

df['Target'] = np.ones((len(df)))
df['Target'].loc[df.Returns > df.Returns.quantile(.80)] = 2
df['Target'].loc[df.Returns < df.Returns.quantile(.20)] = 0

df.drop(['Date', 'Time', 'Open', 'High', 'Low', 'Close', 'Returns', 'MA35', 'MA70', 'MA200'], axis=1, inplace=True)

X = df.drop('Target', axis=1).values
Y = df.Target.values


# convert y into categorical
K = 3
from keras.utils import to_categorical
Y = to_categorical(Y, num_classes=K)

# split train test
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, random_state=42)

print('train shape: ', x_train.shape)
print('test shape: ', x_test.shape)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
Using TensorFlow backend.


train shape:  (135548, 5)
test shape:  (58093, 5)


In [0]:
def softmax(a):
  a = np.exp(a)
  return a / a.sum(axis=1, keepdims=True)

def sigmoid(z):
  return 1 / 1 + np.exp(-z)

def cross_entropy(t, y):
  return - np.mean(t * np.log(y))

def model_score(t, y):
  return np.mean(t == y)

def predict(y):
  return np.argmax(y, axis=1)

# Deep Neural Network using Only Numpy
lr = 0.0000001
epochs = 1000

train_costs = []
test_costs = []

hidden1 = 64
hidden2 = 64
hidden3 = 64
k = 3

w1 = np.random.randn(X.shape[1], hidden1)
b1 = np.random.randn(hidden1)
w2 = np.random.randn(hidden1, hidden2)
b2 = np.random.randn(hidden2)
w3 = np.random.randn(hidden2, hidden3)
b3 = np.random.randn(hidden3)
w4 = np.random.randn(hidden3, k)
b4 = np.random.randn(k)

for i in range(epochs):
  #feed forward train 
  z_train1 = np.tanh(x_train.dot(w1) + b1)
  z_train2 = np.tanh(z_train1.dot(w2) + b2)
  z_train3 = np.tanh(z_train2.dot(w3) + b3)
  y_pred_train = softmax(z_train3.dot(w4) + b4)
  cost_train = cross_entropy(y_train, y_pred_train)
  train_costs.append(cost_train)
  
  if i % 10 == 0:
    print(i, ' cost: ', cost_train)
  
  #feed forward test
  z_test1 = np.tanh(x_test.dot(w1) + b1)
  z_test2 = np.tanh(z_test1.dot(w2) + b2)
  z_test3 = np.tanh(z_test2.dot(w3) + b3)
  y_pred_test = softmax(z_test3.dot(w4) + b4)
  cost_test = cross_entropy(y_test, y_pred_test)
  test_costs.append(cost_test)
  
  #Backpropagation (Gradient descent)
  delta = y_train - y_pred_train
  w4 += z_train3.T.dot(delta) * lr
  b4 += np.sum(delta) * lr
  
  dz3 = delta.dot(w4.T) * (1 - np.power(z_train3, 2))
  
  w3 += z_train2.T.dot(dz3) * lr
  b3 += np.sum(dz3) * lr
  
  dz2 = z_train3.dot(w3.T) * (1 - np.power(z_train2, 2))
  
  w2 += z_train1.T.dot(dz2) * lr
  b2 += np.sum(dz2) * lr
  
  dz = z_train2.dot(w2.T) * (1 - np.power(z_train1, 2))
  
  w1 += x_train.T.dot(dz) * lr
  b1 += np.sum(dz) * lr
  
  
  
plt.plot(train_costs)
plt.plot(test_costs)
plt.show()

print(model_score(predict(y_test), predict(y_pred_test)))

0  cost:  3.554146577500309
10  cost:  1.1629438695221426
20  cost:  0.56915242354883
30  cost:  0.47486401234833386
40  cost:  0.40401978898100266
50  cost:  0.394796285811895
60  cost:  0.36455231080680145
70  cost:  0.3608672100486248
80  cost:  0.4054843320232606
90  cost:  0.3922087734280641
100  cost:  0.37035826114578846
110  cost:  0.33966197590660424
120  cost:  0.3243096211059488
130  cost:  0.32048501784083705
140  cost:  0.31883193361682033
150  cost:  0.31800319610583416
160  cost:  0.3175342296635734
170  cost:  0.31724624672081825
180  cost:  0.3170581241203034
190  cost:  0.31692862649153686
200  cost:  0.31683536435252785
210  cost:  0.3167656304637864
220  cost:  0.3167118826434337
230  cost:  0.316669405861054
240  cost:  0.31663515379250795
250  cost:  0.31660711085685034
260  cost:  0.3165838810579945
270  cost:  0.31656445775223213
280  cost:  0.31654810274650597
290  cost:  0.3165342667475754
300  cost:  0.3165225300418701
310  cost:  0.31651256164106406
320  cos

In [0]:
# use keras for comparision
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

early_stop = tf.keras.callbacks.EarlyStopping(patience=10, monitor='loss', mode='min')

model = Sequential([
    Dense(64, input_shape=(X.shape[1],), activation=tf.nn.tanh),
    Dense(64, activation=tf.nn.tanh),
    Dense(64, activation=tf.nn.tanh),
    Dense(K, activation=tf.nn.softmax)
])
model.compile(optimizer=tf.train.AdamOptimizer(learning_rate=0.000001), loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(x_train, y_train, epochs=1000, callbacks=[early_stop], validation_split=0.3, validation_data=(x_test, y_test), batch_size=32, verbose=1)
model.evaluate(x_final, y_final)

def history_plot(histories, key='acc'):
    plt.figure(figsize=(16,10))

    for name, history in histories:
        val = plt.plot(history.epoch, history.history['val_'+key],
                       '--', label=name.title()+' Val')
        plt.plot(history.epoch, history.history[key], color=val[0].get_color(),
                 label=name.title()+' Train')

        plt.xlabel('Epochs')
        plt.ylabel(key.replace('_',' ').title())
        plt.legend()

        plt.xlim([0,max(history.epoch)])
        
history_plot(('model', history))